<a href="https://colab.research.google.com/github/fender8185/Stock-predict/blob/main/lstm01stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install FinMind==1.5.4

In [12]:
import tensorflow as tf
import numpy as np
import FinMind
from FinMind.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from datetime import date


In [13]:
params = {
        'stock_id': 2330,
        'start_date': '2018-01-01',
        'end_date': date.today()

    }

data_loader = DataLoader()
df_holding = data_loader.taiwan_stock_holding_shares_per(**params)
df_holding = df_holding.pivot(index='date', columns='HoldingSharesLevel', values='percent').reset_index()
stock_data = data_loader.taiwan_stock_daily(**params)
df = df_holding.merge(stock_data, on='date', how='outer')
selected_columns = ['date', '1,000-5,000', '1-999', '10,001-15,000', '100,001-200,000',
       '15,001-20,000', '20,001-30,000', '200,001-400,000', '30,001-40,000',
       '40,001-50,000', '400,001-600,000', '5,001-10,000', '50,001-100,000',
       '600,001-800,000', '800,001-1,000,000', 'more than 1,000,001', 'close']

# Remove rows where all shareholder ratios are NaN or 0
df = df[selected_columns]

df[[ 'close']] = df[[ 'close']].replace(0, np.nan).fillna(method='ffill')

valid_ratio_indices = df[df.columns.difference(['date', 'close'])].dropna(how='all').index
df = df.loc[valid_ratio_indices]
all_dates = df['date'].values[1:]
df.drop(columns=['date'], inplace=True)
print(df)
numpy_array = df.values


     1,000-5,000  1-999  10,001-15,000  100,001-200,000  15,001-20,000  \
0           0.99   0.12           0.40             0.80           0.27   
1           0.98   0.12           0.40             0.80           0.27   
2           0.95   0.12           0.39             0.80           0.26   
3           0.94   0.12           0.39             0.80           0.26   
4           0.95   0.12           0.39             0.80           0.26   
..           ...    ...            ...              ...            ...   
286         2.75   0.52           0.70             1.00           0.47   
287         2.78   0.53           0.70             1.00           0.47   
288         2.80   0.53           0.70             1.01           0.47   
289         2.82   0.54           0.71             1.01           0.48   
290         2.85   0.54           0.71             1.01           0.48   

     20,001-30,000  200,001-400,000  30,001-40,000  40,001-50,000  \
0             0.40             1.07       

In [ ]:
# 計算價差
price_diff = numpy_array[1:, -1] - numpy_array[:-1, -1]
# 更新 y
y = price_diff
# X 仍然是前一天的特徵值，但需要去掉最後一天的數據，以確保與 y 的長度相同
X = numpy_array[:-2,:-1]

split_index = int(len(X) * 0.8)  # 80% 資料作為訓練集
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]
train_dates = all_dates[:split_index]
test_dates = all_dates[split_index+1:]  # 因為 y 是從第二天開始的，所以 test_dates 也需要從第二天開始

# Reshape data for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

Epoch 1/100
8/8 [==============================] - 6s 23ms/step - loss: 193.3325
Epoch 2/100
8/8 [==============================] - 0s 22ms/step - loss: 192.5841
Epoch 3/100
8/8 [==============================] - 0s 23ms/step - loss: 192.0970
Epoch 4/100
8/8 [==============================] - 0s 22ms/step - loss: 192.8983
Epoch 5/100
8/8 [==============================] - 0s 23ms/step - loss: 192.0050
Epoch 6/100
8/8 [==============================] - 0s 23ms/step - loss: 191.9829
Epoch 7/100
8/8 [==============================] - 0s 22ms/step - loss: 192.0978
Epoch 8/100
8/8 [==============================] - 0s 24ms/step - loss: 192.1316
Epoch 9/100
8/8 [==============================] - 0s 22ms/step - loss: 192.0886
Epoch 10/100
8/8 [==============================] - 0s 21ms/step - loss: 192.0601
Epoch 11/100
8/8 [==============================] - 0s 23ms/step - loss: 192.0189
Epoch 12/100
8/8 [==============================] - 0s 22ms/step - loss: 191.9730
Epoch 13/100
8/8 [=======

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Data', 'Testing Data with Predictions'))

fig.add_trace(go.Scatter(x=train_dates, y=y_train, mode='lines', name='Training Data'), row=1, col=1)
fig.add_trace(go.Scatter(x=train_dates, y=y_pred_train.flatten(), mode='lines', name='Predicted Training Data', marker=dict(symbol='circle-open')), row=1, col=1)

fig.add_trace(go.Scatter(x=test_dates, y=y_test, mode='lines', name='Real Test Values'), row=1, col=2)
fig.add_trace(go.Scatter(x=test_dates, y=y_pred_test.flatten(), mode='lines', name='Predicted Test Values', marker=dict(symbol='circle-open')), row=1, col=2)

fig.update_layout(hovermode='x unified')
fig.show()
